# Text to speech

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install azure-cognitiveservices-speech

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.2 MB/s 


In [ ]:
!pip install --upgrade azure-cognitiveservices-speech


In [ ]:
import azure.cognitiveservices.speech as speechsdk


In [ ]:
from azure.cognitiveservices.speech import (
    AudioDataStream,
    SpeechConfig,
    SpeechSynthesizer,
    SpeechSynthesisOutputFormat,
)

In [ ]:
API_KEY='070e2498f4aa451d9fa29fc493abf53b'

In [ ]:
speech_config = SpeechConfig(subscription=API_KEY, region="eastus")

In [ ]:
speech_config = speechsdk.SpeechConfig(subscription=API_KEY,
                                       region="eastus",
                                       speech_recognition_language='en-GB')

In [ ]:
audio_format = "Riff24Khz16BitMonoPcm"

In [ ]:
speech_config.set_speech_synthesis_output_format(
    SpeechSynthesisOutputFormat[audio_format]
)

In [ ]:
with open('input.xml', 'r') as f:
    data = f.read()

In [ ]:
import random
from azure.cognitiveservices.speech import (
    AudioDataStream,
    SpeechConfig,
    SpeechSynthesizer,
    SpeechSynthesisOutputFormat,
)

# Subscription settings from Azure
# Region can be westeurope for example
key = '070e2498f4aa451d9fa29fc493abf53b'
region = "eastus"
subscription_key = '070e2498f4aa451d9fa29fc493abf53b'
subscription_region = "eastus"

# Input SSML file
# Open this file to change or fine-tune the pitch, pronunciation, speaking rate, volume, voice, language and more
# https://github.com/MicrosoftDocs/azure-docs/blob/master/articles/cognitive-services/Speech-Service/language-support.md#neural-voices

input_file = "input.xml"

# https://docs.microsoft.com/nl-nl/python/api/azure-cognitiveservices-speech/azure.cognitiveservices.speech.speechsynthesisoutputformat?view=azure-python
audio_format = "Riff24Khz16BitMonoPcm"

# Output folder and file
output_folder = "/content/drive/MyDrive/Data text to speech"
output_file = f"file-{random.randint(10000,99999)}.wav"

speech_config = SpeechConfig(subscription=subscription_key, region=subscription_region)
speech_config.set_speech_synthesis_output_format(
    SpeechSynthesisOutputFormat[audio_format]
)
synthesizer = SpeechSynthesizer(speech_config=speech_config, audio_config=None)

input = open(f"{input_file}", "r").read()
result = synthesizer.speak_ssml_async(input).get()

stream = AudioDataStream(result)
stream.save_to_wav_file(f"{output_file}")

# Speech to text

In [ ]:
import glob
import azure.cognitiveservices.speech as speechsdk
import time
import pandas as pd

In [ ]:
speech_key = "070e2498f4aa451d9fa29fc493abf53b"
service_region = "eastus"

# File location
location = "/content/drive/MyDrive/Data speech to text"

In [ ]:
speech_key, service_region = speech_key, service_region
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

In [ ]:
def speech_recognize_continuous_from_file(file):
    """performs continuous speech recognition with input from an audio file"""
    # <SpeechContinuousRecognitionWithFile>
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    audio_config = speechsdk.audio.AudioConfig(filename=file)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False

    def stop_cb(evt):
        """callback that stops continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True

    all_results = []
    def handle_final_result(evt):
        all_results.append(evt.result.text)

    speech_recognizer.recognized.connect(handle_final_result)
    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    print("Printing all results:")
    print(all_results)

    df = pd.DataFrame(all_results)
    df

    file_name = file + r"-speech-to-text-csv-output.csv"
    df.to_csv(file_name)


    print ("Audio File: "+file+" converted successfully")
    print ("####################################################################################")


In [ ]:
location = location

fileset = [file for file in glob.glob(location + "**/*.wav", recursive=True)]

# Loop to call function to convert audio files to text
for file in fileset:
    #run_speech_to_text_small_audio_files(file)
    speech_recognize_continuous_from_file(file)
    print(file)

SESSION STARTED: SessionEventArgs(session_id=1c855b2f8232431fad540f29ea9e780b)
RECOGNIZING: SpeechRecognitionEventArgs(session_id=1c855b2f8232431fad540f29ea9e780b, result=SpeechRecognitionResult(result_id=53c2ad079a744c5ebe55e1733bb76c80, text="don't ask me", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=1c855b2f8232431fad540f29ea9e780b, result=SpeechRecognitionResult(result_id=99eab42008da4cf786e907595e8bd950, text="don't ask me to", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=1c855b2f8232431fad540f29ea9e780b, result=SpeechRecognitionResult(result_id=f7117aa892574eb8b896ba9fe5eb48ca, text="don't ask me to carry an oily", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=1c855b2f8232431fad540f29ea9e780b, result=SpeechRecognitionResult(result_id=56ad066d0aed4d74bca509abb603dfca, text="don't ask me to carry an oily rag like", reason=ResultReason.Recognizing